In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from model import AdderNet, AdderNet2_0
from Train import model_training
from quantization_encoder import Quant
from quantization_decoding import Quant_decode

In [3]:
# Hyper parameters
epoch_losses = []
train_loss_list = []
lr = 0.001
epochs = 200
train_batch_size = 256
test_batch_size = 128
in_channels = 3
out_channels = 16
kernel_size = 3

# Model device assignment
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = AdderNet().to(device)

In [ ]:
# Define transformation
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                        (0.2023, 0.1994, 0.2010))
])

# Test transform (no augmentation)
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                        (0.2023, 0.1994, 0.2010))
])

# Load the training dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=train_transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=train_batch_size,
                                          shuffle=True, num_workers=8)

# Load the test dataset
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=test_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=test_batch_size,
                                         shuffle=False, num_workers=0)

# Define the class names for reference
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

  9%|▉         | 15.9M/170M [00:54<08:25, 306kB/s]

In [ ]:
# Training AdderNet model on Cifar10
train_model = model_training(model, lr, trainloader, epochs)
training_loss = train_model.forward(device)

In [ ]:
# Visualization of training results
plt.plot(training_loss, linewidth=2, c='r')
plt.xlabel("Epochs")
plt.ylabel("Training Loss")
plt.title("Training Loss of Addernet Model")
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:

model.eval()
with torch.no_grad():
    number_correct = 0
    samples = 0
    for images, labels in testloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)

        _, prediction = torch.max(outputs, 1)
        samples += labels.size(0)
        number_correct += (prediction == labels).sum().item()

accuracy = (number_correct / samples) * 100.0
print(f"Accuracy: {accuracy:.2f}")

In [ ]:
# Save the model
torch.save(model.state_dict(), 'AdderNet_model.pth')
print("Model saved as 'AdderNet_model.pth'")


In [ ]:
# Editing .pth state dictionary
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

model_state_dict = torch.load('AdderNet_model.pth', map_location=device)
fixed_state_dict = {}

def remap_key(key: str) -> str:
    """Map original checkpoint keys to the correct AdderNet naming."""
    new_key = key.replace('module.', '')

    # conv and batchnorm
    if new_key.startswith('conv1.') or new_key.startswith('bn1.') or new_key.startswith('fc.') or new_key.startswith('bn2.'):
        return new_key

    # Process residual layers
    for layer_num in [1, 2, 3]:
        prefix = f'layer{layer_num}.'
        if new_key.startswith(prefix):
            rest = new_key[len(prefix):]  # everything after 'layerX.'

            # If next is block index
            if rest[0].isdigit():
                block_num = rest[0]
                rest_after_block = rest[2:]

                # Handle downsample case
                if rest_after_block.startswith('downsample.'):
                    ds_rest = rest_after_block[len('downsample.'):]
                    if ds_rest.startswith('0.'):
                        return f'layer{layer_num}.downsample_adder.{ds_rest[2:]}'
                    elif ds_rest.startswith('1.'):
                        return f'layer{layer_num}.downsample_bn.{ds_rest[2:]}'

                # Otherwise: normal residual block conv/bn
                rest_after_block = rest_after_block.replace('conv1.', 'adder1.')
                rest_after_block = rest_after_block.replace('conv2.', 'adder2.')
                return f'layer{layer_num}.blocks.{block_num}.{rest_after_block}'

    # If nothing matches, just return
    return new_key

# Apply the mapping
for key, value in model_state_dict.items():
    fixed_key = remap_key(key)
    fixed_state_dict[fixed_key] = value

params = list(fixed_state_dict.items())
n_params = len(params)

In [ ]:
n_cols = 16  # 16 columns
n_rows = (n_params + n_cols - 1) // n_cols  # 8 rows for 128 params

# Create subplots with larger figure
fig, axes = plt.subplots(n_rows, n_cols, figsize=(50, 25))
axes = axes.flatten()

# Plot each parameter
for i, (name, tensor) in enumerate(params):
    values = tensor.cpu().numpy().flatten()
    axes[i].hist(values, bins=50)
    axes[i].set_title(name)
    axes[i].grid(True, alpha=0.3)


plt.tight_layout()
plt.show()

In [ ]:
filtered_params = []
for name, tensor in params:
    # Include if name contains: weight, bias, adder
    if any(keyword in name.lower() for keyword in ['weight', 'bias', 'adder']):
        filtered_params.append((name, tensor))

n_params = len(filtered_params)

n_cols = 5  # 5 columns
n_rows = (n_params + n_cols - 1) // n_cols  # 13 rows for 65 params

# Create subplots with larger figure
fig, axes = plt.subplots(n_rows, n_cols, figsize=(50, 25))
axes = axes.flatten()

# Plot each parameter
for i, (name, tensor) in enumerate(filtered_params):
    values = tensor.cpu().numpy().flatten()
    axes[i].hist(values, bins=50)
    axes[i].set_title(name)
    axes[i].grid(True, alpha=0.3)


plt.tight_layout()
plt.show()

In [ ]:
# Quantization of weight tensors for AdderNet
quant_acc = []
quant_error = []
bit_array = [16, 14, 12, 10, 8, 5, 6, 4, 3]
activations_dict = {}

for i, bits in enumerate(bit_array):
  activations_dict[bits] = {
    'input_activation': None,
    'prelayer_activation': None,
    'layer1_activation': None,
    'layer2_activation': None,
    'layer3_activation': None
  }
  quant_dict = {}
  max_error = 0.0

  for name, weight_tensor in params:
    quantized_tensor, scale = Quant.symmetric_quantization(weight_tensor, bits=bits)
    fp32_tensor, error = Quant_decode.dequant(scale, quantized_tensor, weight_tensor)
    quant_dict[name] = fp32_tensor.to(device)
    max_error = max(max_error, error)

  quant_model = AdderNet().to(device)
  quant_model.load_manual_weights(quant_dict)
  quant_model.eval()

  print(f"Testing {bits}-bit quantization")
  with torch.no_grad():
      number_correct = 0
      samples = 0
      first_batch = True
      for images, labels in testloader:
          images = images.to(device)
          labels = labels.to(device)
          outputs = quant_model(images, save_activations=True)

          _, prediction = torch.max(outputs, 1)
          samples += labels.size(0)
          number_correct += (prediction == labels).sum().item()

          if first_batch:
            for name, activation in quant_model.activations.items():
              activations_dict[bits][name] = activation.cpu().clone()
            first_batch = False

  del quant_model
  torch.cuda.empty_cache() if torch.cuda.is_available() else None

  accuracy = (number_correct / samples) * 100.0
  quant_acc.append(accuracy)
  print(f"  Accuracy: {accuracy:.2f}%")
  quant_error.append(max_error)


In [ ]:
# AOQ of weight tensors for AdderNet2.0
quant_acc_2_0 = []
bit_array = [16, 14, 12, 10, 8, 6, 5, 4, 3]
activations_dict2_0 = {}
clipped_weight_dict = {}
clipped_bias_dict = {}

for i, bits in enumerate(bit_array):
  activations_dict2_0[bits] = {
    'input_activation_2.0': None,
    'prelayer_activation_2.0': None,
    'layer1_activation_2.0': None,
    'layer2_activation_2.0': None,
    'layer3_activation_2.0': None
  }

  quantized_state_dict = {k: v.clone() for k, v in fixed_state_dict.items()}

  max_val = 2.5
  Max_A = 2**(bits-1) - 1      # Maximum value: 127 for bits=8
  Max_B = -1 * 2**(bits-1)     # Minimum value: -128 for bits=8
  delta = max_val / (2**(bits-1) - 1)

  print(f"\nProcessing {bits}-bit quantization with FBR. Delta Val: {delta}")

  x_tensor = quantized_state_dict['bn1.weight']
  xq_tensor = x_tensor / delta
  quantized_state_dict['bn1.weight'] = xq_tensor

  x_tensor = quantized_state_dict['bn1.bias']
  xq_tensor = x_tensor / delta
  quantized_state_dict['bn1.bias'] = xq_tensor

  clipped_bias_dict[bits] = {}
  clipped_weight_dict[bits] = {}

  for name in quantized_state_dict.keys():
      if name.startswith('layer'):
          if name.endswith('adder'):
              w_tensor = quantized_state_dict[name]
              wq = torch.round(w_tensor/delta)
              wq_clamp = torch.clamp(wq, max=Max_A, min=Max_B)
              quantized_state_dict[name] = wq_clamp
              bias_tensor = (wq - wq_clamp).abs()
              bias_sum = torch.sum(bias_tensor, dim=(1,2,3))
              clipped_weight_dict[bits][name] = bias_sum.cpu().clone()
              clipped_bias_dict[bits][name.replace('adder', 'bn')] = bias_sum.cpu().clone()
          if name.endswith('running_mean'):
              m_tensor = quantized_state_dict[name]
              mq = torch.round(m_tensor/delta)
              quantized_state_dict[name] = mq + bias_sum
          if name.endswith('bias'):
              x_tensor=quantized_state_dict[name]
              xq_tensor = x_tensor/delta
              quantized_state_dict[name] = xq_tensor

  quantized_state_dict['fc.weight']=quantized_state_dict['fc.weight']*delta

  quant_model = AdderNet2_0(num_classes=10).to(device)
  quant_model.load_manual_weights(quantized_state_dict)
  quant_model.eval()

  print(f"Testing {bits}-bit quantization AdderNet2.0")
  with torch.no_grad():
      number_correct = 0
      samples = 0
      first_batch = True
      for images, labels in testloader:
          images = images.to(device)
          labels = labels.to(device)
          outputs = quant_model(images, save_activations=True)

          _, prediction = torch.max(outputs, 1)
          samples += labels.size(0)
          number_correct += (prediction == labels).sum().item()

          if first_batch:
            for name, activation in quant_model.activations.items():
              activations_dict2_0[bits][name] = activation.cpu().clone()
            first_batch = False

  accuracy = (number_correct / samples) * 100.0
  print(f"  Accuracy: {accuracy:.2f}%")

  del quant_model
  torch.cuda.empty_cache() if torch.cuda.is_available() else None
  quant_acc_2_0.append(accuracy)

In [ ]:
# AOQ of unsigned weight tensors for AdderNet2.0
quant_acc_2_0_U = []
bit_array = [16, 14, 12, 10, 8, 6, 5, 4, 3]
activations_dict2_0_U = {}
clipped_bias_dict_U = {}
clipped_weight_dict_U = {}

for i, bits in enumerate(bit_array):
  activations_dict2_0_U[bits] = {
    'input_activation_2.0': None,
    'prelayer_activation_2.0': None,
    'layer1_activation_2.0': None,
    'layer2_activation_2.0': None,
    'layer3_activation_2.0': None
  }

  quantized_state_dict = {k: v.clone() for k, v in fixed_state_dict.items()}

  max_val = 2.5
  Max_A = 2**(bits) - 1      # Maximum value: 127 for bits=8
  Max_B = 0                    # Minimum value: 0 for unsigned
  delta = max_val / (Max_A)

  print(f"\nProcessing {bits}-bit quantization with FBR. Delta Val: {delta}")

  x_tensor = quantized_state_dict['bn1.weight']
  xq_tensor = x_tensor / delta
  quantized_state_dict['bn1.weight'] = xq_tensor

  x_tensor = quantized_state_dict['bn1.bias']
  xq_tensor = x_tensor / delta
  quantized_state_dict['bn1.bias'] = xq_tensor

  clipped_bias_dict_U[bits] = {}
  clipped_weight_dict_U[bits] = {}

  for name in quantized_state_dict.keys():
      if name.startswith('layer'):
          if name.endswith('adder'):
              w_tensor = quantized_state_dict[name]
              wq = torch.round(w_tensor/delta)
              wq_clamp = torch.clamp(wq, max=Max_A, min=Max_B)
              quantized_state_dict[name] = wq_clamp
              bias_tensor = (wq - wq_clamp).abs()
              bias_sum = torch.sum(bias_tensor, dim=(1,2,3))
              clipped_weight_dict_U[bits][name] = bias_sum.cpu().clone()
              clipped_bias_dict_U[bits][name.replace('adder', 'bn')] = bias_sum.cpu().clone()
          if name.endswith('running_mean'):
              m_tensor = quantized_state_dict[name]
              mq = torch.round(m_tensor/delta)
              quantized_state_dict[name] = mq + bias_sum
          if name.endswith('bias'):
              x_tensor=quantized_state_dict[name]
              xq_tensor = x_tensor/delta
              quantized_state_dict[name] = xq_tensor

  quantized_state_dict['fc.weight']=quantized_state_dict['fc.weight']*delta

  quant_model = AdderNet2_0(num_classes=10).to(device)
  quant_model.load_manual_weights(quantized_state_dict)
  quant_model.eval()

  print(f"Testing {bits}-bit quantization AdderNet2.0")
  with torch.no_grad():
      number_correct = 0
      samples = 0
      first_batch = True
      for images, labels in testloader:
          images = images.to(device)
          labels = labels.to(device)
          outputs = quant_model(images, save_activations=True)

          _, prediction = torch.max(outputs, 1)
          samples += labels.size(0)
          number_correct += (prediction == labels).sum().item()

          if first_batch:
            for name, activation in quant_model.activations.items():
              activations_dict2_0_U[bits][name] = activation.cpu().clone() # Fixed: Changed to activations_dict2_0_U
            first_batch = False

  accuracy = (number_correct / samples) * 100.0
  print(f"  Accuracy: {accuracy:.2f}%")

  del quant_model
  torch.cuda.empty_cache() if torch.cuda.is_available() else None
  quant_acc_2_0_U.append(accuracy)


In [ ]:
for bits in clipped_bias_dict.keys():
    # Plot bias distribution for this bit width
    all_biases = torch.cat([v.flatten() for v in clipped_bias_dict[bits].values()]).cpu().numpy()

    # Plot weight distribution for this bit width
    all_weights = torch.cat([v.flatten() for v in clipped_weight_dict[bits].values()]).cpu().numpy()

    plt.figure(figsize=(10, 4))

    plt.subplot(1, 2, 1)
    plt.hist(all_biases, bins=50)
    plt.title(f'Clipping Bias Distribution ({bits}-bit)')
    plt.xlabel('Bias Value')
    plt.ylabel('Frequency')

    plt.subplot(1, 2, 2)
    plt.hist(all_weights, bins=100)
    plt.title(f'Quantized Weight Distribution ({bits}-bit)')
    plt.xlabel('Weight Value')
    plt.ylabel('Frequency')

    plt.tight_layout()
    plt.show()

In [ ]:
for bits in clipped_bias_dict_U.keys():
    # Plot bias distribution for this bit width
    all_biases = torch.cat([v.flatten() for v in clipped_bias_dict_U[bits].values()]).cpu().numpy()

    # Plot weight distribution for this bit width
    all_weights = torch.cat([v.flatten() for v in clipped_weight_dict_U[bits].values()]).cpu().numpy()

    plt.figure(figsize=(10, 4))

    plt.subplot(1, 2, 1)
    plt.hist(all_biases, bins=50)
    plt.title(f'Clipping Bias Distribution ({bits}-bit)')
    plt.xlabel('Bias Value')
    plt.ylabel('Frequency')

    plt.subplot(1, 2, 2)
    plt.hist(all_weights, bins=100)
    plt.title(f'Quantized Weight Distribution ({bits}-bit)')
    plt.xlabel('Weight Value')
    plt.ylabel('Frequency')

    plt.tight_layout()
    plt.show()

In [ ]:
activation_names = list(activations_dict[16].keys())
num_bits = len(bit_array)
num_activations = len(activation_names)

fig, axes = plt.subplots(num_bits, num_activations, figsize=(20, 25))

for i, bits in enumerate(bit_array):
  for j, name in enumerate(activation_names):
    ax = axes[i, j]

    act_values = activations_dict[bits][name].cpu().flatten()

    ax.hist(act_values, bins=50, edgecolor='black', alpha=0.7)
    ax.set_title(f'{bits}-bit: {name}')
    ax.set_xlabel('Activation Value')
    ax.set_ylabel('Frequency')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
activation_names = list(activations_dict2_0[16].keys())
num_bits = len(bit_array)
num_activations = len(activation_names)

fig, axes = plt.subplots(num_bits, num_activations, figsize=(20, 25))

for i, bits in enumerate(bit_array):
  for j, name in enumerate(activation_names):
    ax = axes[i, j]

    act_values = activations_dict2_0[bits][name].cpu().flatten()

    ax.hist(act_values, bins=50, edgecolor='black', alpha=0.7)
    ax.set_title(f'{bits}-bit AdderNet2.0: {name}')
    ax.set_xlabel('Activation Value')
    ax.set_ylabel('Frequency')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
activation_names = list(activations_dict2_0_U[16].keys())
num_bits = len(bit_array)
num_activations = len(activation_names)

fig, axes = plt.subplots(num_bits, num_activations, figsize=(20, 25))

for i, bits in enumerate(bit_array):
  for j, name in enumerate(activation_names):
    ax = axes[i, j]

    act_values = activations_dict2_0_U[bits][name].cpu().flatten()

    ax.hist(act_values, bins=50, edgecolor='black', alpha=0.7)
    ax.set_title(f'{bits}-bit AdderNet2.0: {name}')
    ax.set_xlabel('Activation Value')
    ax.set_ylabel('Frequency')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

ax.scatter(bit_array, quant_acc, s=80, c='green', alpha=0.7, label='AdderNet (Original)')
ax.plot(bit_array, quant_acc, linewidth=2, c='green', linestyle='--')
ax.scatter(bit_array, quant_acc_2_0, s=80, c='blue', alpha=0.7, label='AdderNet2.0')
ax.plot(bit_array, quant_acc_2_0, linewidth=2, c='r')
ax.scatter(bit_array, quant_acc_2_0_U, s=80, c='red', alpha=0.7, label='AdderNet2.0 Unsigned')
ax.plot(bit_array, quant_acc_2_0_U, linewidth=2, c='r')

ax.set_title('Bitwidth vs. Accuracy')
ax.set_xlabel('BitWidth')
ax.set_ylabel('Accuracy (%)')
ax.legend(fontsize=11)
ax.grid(True)
ax.invert_xaxis()

plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

ax.scatter(bit_array, quant_error, s=80, c='green', alpha=0.7, label='Testing Datapoints')
ax.plot(bit_array, quant_error, linewidth=2, c='r')
ax.set_title('Bitwidth vs. Quantization Error')
ax.set_xlabel('BitWidth')
ax.set_ylabel('Error')
ax.legend(fontsize=11)
ax.grid(True)

plt.tight_layout()
plt.show()